<a href="https://colab.research.google.com/github/prateekrana17/SPARK-NLP-RAG-LLM/blob/main/SPARK-NLP-latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh


--2024-05-15 10:00:55--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘colab_setup.sh.2’

colab_setup.sh.2    100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-05-15 10:00:55 (36.7 MB/s) - ‘colab_setup.sh.2’ saved [1191/1191]



In [2]:
!bash colab_setup.sh

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


In [3]:
!pip install spark-nlp-display

In [4]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [5]:
spark = sparknlp.start()
spark


In [6]:
# detection model_name = 'ld_wiki_cnn_231'
model_name = 'ld_wiki_tatoeba_cnn_375'

In [7]:
text = """Die Mona Lisa ist ein von Leonardo geschaffenes Ölgemälde aus dem 16. Jahrhundert. Sie findet im Louvre in Paris statt."""  #document/ ECM retireved text

text2 = """Ich liebe Datenwissenschaf"""

In [8]:
# create spark data frame

data = spark.createDataFrame([
    [text],
    [text2]]).toDF("text")

In [9]:
# detection pipeline for 100+ languages

# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)


documentAssembler = DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")

sentence_detector = SentenceDetector() \
            .setInputCols(["document"]) \
            .setOutputCol("sentence")

# Step 2: Determines the language of the text
languageDetector = LanguageDetectorDL.pretrained(model_name)\
            .setInputCols("sentence")\
            .setOutputCol("language")\
            .setThreshold(0.5)\
            .setCoalesceSentences(True)

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentence_detector,
        languageDetector])


ld_wiki_tatoeba_cnn_375 download started this may take some time.
Approximate size to download 8.8 MB
[OK!]


In [10]:
#language detection
result = nlpPipeline.fit(data).transform(data)
result.select("text", "language.result").show()


+--------------------+------+
|                text|result|
+--------------------+------+
|Die Mona Lisa ist...|  [de]|
|Ich liebe Datenwi...|  [de]|
+--------------------+------+



In [11]:
# prompt: fetch the item inside the 'result' column in the dataframe above and store the values in variable d1 and d2

language1 = result.select("text", "language.result").first()[1]
language2 = result.select("text", "language.result").take(2)[1][1]

print(language1)
print(language2)

['de']
['de']


In [44]:
# Create Spark DataFrame
data = spark.createDataFrame([[text], [text2]]).toDF("text")
data.show()


+--------------------+
|                text|
+--------------------+
|Die Mona Lisa ist...|
|Ich liebe Datenwi...|
+--------------------+



In [13]:
# Translation pipeline
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

## More accurate Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")

marian = MarianTransformer.pretrained("opus_mt_it_en", "xx").setInputCols(["sentences"]).setOutputCol("translation")  # add f string variable here to enable translation to language of choice



nlp_pipeline = Pipeline(
    stages=[
        documentAssembler,
        sentencerDL,
        marian
        ])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_it_en download started this may take some time.
Approximate size to download 454.8 MB
[OK!]


In [40]:
!pip install PyDrive

In [45]:
# PyDrive Libraries
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [46]:
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [49]:
#open a file and read the text from it into a string variable

# Function to read Google Docs file
def read_google_doc(file_id):
    file = drive.CreateFile({'id': file_id})
    file.FetchMetadata(fields='title, mimeType')
    download_mime_type = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
    file.GetContentFile('temp.docx', mimetype=download_mime_type)

    # Convert docx to text
    import docx
    doc = docx.Document('temp.docx')
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Replace 'your_file_id' with the actual file ID of your Google Docs file
file_id = '1Bh152c2kKbzUe7ZMquL0-juKHj_wdswZ3674UAZlUKU'
doc_content = read_google_doc(file_id)
print(doc_content)







FileNotDownloadableError: No downloadLink/exportLinks for mimetype found in metadata

In [37]:
path = '/content/drive/MyDrive/cc/german_example.gdoc'  #replace with API call

# colab only code ends


In [50]:
!ls /content/drive/MyDrive/cc/

 app1.py   app2.py   german_example.gdoc   input   README.md  'Untitled project.gscript'


In [39]:
#open a file and read the text from it into a string variable   #replace with API call
with open(path, 'r') as file:
  print(file.read())
  translation_text = file.read()

OSError: [Errno 95] Operation not supported: '/content/drive/MyDrive/cc/german_example.gdoc'

In [ ]:
#create spark Data Frame for displaying detection results
df = spark.createDataFrame(user_text, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

In [ ]:
# Tranlation pipeline


## More accurate Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")


marian = MarianTransformer.pretrained("opus_mt_it_en", "xx")\
  .setInputCols(["sentences"])\
  .setOutputCol("translation")

nlp_pipeline2 = Pipeline(
    stages=[
        documentAssembler,
        sentencerDL,
        marian
        ])

In [ ]:
# translation dataframe

empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = nlp_pipeline2.fit(empty_df)
lmodel = LightPipeline(pipeline_model)
res = lmodel.fullAnnotate(translation_text)
